In [1]:
#  *** THIS DATA (FROM API) HAS NOT BEEN CHECKED AGAINST DATA.CENSUS.GOV TABLES***

In [2]:
## Build an equitable economy: INCOME
### U.S. median income, Black median income, Black-white income gap
## 2019

In [3]:
# see https://jtleider.github.io/censusdata/api.html

In [4]:
# data source: 2019 Census Data (ACS-1 year) for Selected Variables - U.S.

In [5]:
# Dependencies
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from config import api_key

#c = Census(api_key)

In [6]:
import os

In [7]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis\\GOAL-1'

In [8]:
# need to change the working directory; be sure that the config file for the census API is also in this folder
#os.chdir('C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\analysis_nbs')

In [9]:
pd.set_option('display.max_rows', 200)

In [10]:
pd.set_option('max_colwidth', 100)

In [11]:
# see https://iqss.github.io/datafest-project/optional-u-s-census-data-python.html
# see census api example urls     https://api.census.gov/data/2019/acs/acs1/examples.html  
url = f"https://api.census.gov/data/2019/acs/acs1?get=NAME,B19001_001E,B01003_001E,B19013_001E,B02001_002E,B02001_003E,B19013A_001E,B19013B_001E&for=us:1&key={api_key}"

response = requests.get(url)
response.status_code

200

In [12]:
us = response.json()
us_df_income1 = pd.DataFrame(us[1:], columns=us[0])
# alternatively   us_df = pd.DataFrame(us[1:], columns=us[0]).rename(columns={'NAME':'country', 
# B01003_001E": "Population","B01002_001E": "Median Age"})
us_df_income1 

,NAME,B19001_001E,B01003_001E,B19013_001E,B02001_002E,B02001_003E,B19013A_001E,B19013B_001E,us
0,United States,122802852,328239523,65712,236475401,41989671,69823,43862,1


In [13]:

us_df_income1 = us_df_income1.rename(columns={"B19001_001E": "Total households",
                              "B01003_001E": "Population",
                              "B19013_001E": "Median household income",
                              "B02001_003E": "Pop. Black",
                              "B02001_002E": "Pop. white",
                              "B19013A_001E": "Median hh income white",
                              "B19013B_001E": "Median hh income Black",  
                              "NAME": "Location"})
us_df_income1

,Location,Total households,Population,Median household income,Pop. white,Pop. Black,Median hh income white,Median hh income Black,us
0,United States,122802852,328239523,65712,236475401,41989671,69823,43862,1


In [14]:
us_df_income2 = us_df_income1.copy()
us_df_income2

,Location,Total households,Population,Median household income,Pop. white,Pop. Black,Median hh income white,Median hh income Black,us
0,United States,122802852,328239523,65712,236475401,41989671,69823,43862,1


In [15]:
us_df_income = us_df_income2.drop(columns=['us'])

In [16]:
# use .insert() to add a column constant value for year
us_df_income.insert(1,'Year','2019')
us_df_income

,Location,Year,Total households,Population,Median household income,Pop. white,Pop. Black,Median hh income white,Median hh income Black
0,United States,2019,122802852,328239523,65712,236475401,41989671,69823,43862


In [17]:
us_df_income = us_df_income.loc[:,['Location','Year','Population','Pop. Black','Pop. white','Median household income','Median hh income Black','Median hh income white']]
us_df_income

,Location,Year,Population,Pop. Black,Pop. white,Median household income,Median hh income Black,Median hh income white
0,United States,2019,328239523,41989671,236475401,65712,43862,69823


In [18]:
us_df_income.dtypes

Location                   object
Year                       object
Population                 object
Pop. Black                 object
Pop. white                 object
Median household income    object
Median hh income Black     object
Median hh income white     object
dtype: object

In [19]:
us_df_income[['Population','Pop. Black','Pop. white','Median household income','Median hh income Black',
              'Median hh income white']] = us_df_income[['Population','Pop. Black','Pop. white','Median household income','Median hh income Black','Median hh income white']].apply(pd.to_numeric)
us_df_income.dtypes

Location                   object
Year                       object
Population                  int64
Pop. Black                  int64
Pop. white                  int64
Median household income     int64
Median hh income Black      int64
Median hh income white      int64
dtype: object

In [20]:
# Export file as a CSV, without the Pandas index, but with the header
us_df_income.to_csv("us_income_2019.csv", index = False, header=True)

In [21]:
# data to be used to calculate poverty rate, labor participation rate, unemployment rate
url_rate = f"https://api.census.gov/data/2019/acs/acs1?get=NAME,B17001_001E,B17001_002E,B23025_001E,B23025_002E,B23025_004E,B23025_005E&for=us:1&key={api_key}"

response = requests.get(url_rate)
response.status_code

200

In [22]:
us_rates = response.json()
us_df_rates2 = pd.DataFrame(us_rates[1:], columns=us_rates[0])
# alternatively   us_df = pd.DataFrame(us[1:], columns=us[0]).rename(columns={'NAME':'country', 
# B01003_001E": "Population","B01002_001E": "Median Age"})
us_df_rates2  

,NAME,B17001_001E,B17001_002E,B23025_001E,B23025_002E,B23025_004E,B23025_005E,us
0,United States,320118791,39490096,263534161,167501734,158758794,7515579,1


In [23]:
us_df_rates = us_df_rates2.rename(columns={
                                      "B17001_001E": "Persons for whom poverty status determined",    
                                      "B17001_002E": "Individuals- income below poverty level",# universe: persons for whom poverty status is determined
                                      "B23025_001E":  "Population 16 years and over",
                                      "B23025_002E": "In labor force", # universe: Population 16 years and over
                                      "B23025_004E": "Employed civilians", # universe: Population 16 years and over
                                      "B23025_005E": "Unemployed civilians", # universe: Population 16 years and over
                                      "NAME": "Location"
                                      })
us_df_rates.head()

,Location,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,us
0,United States,320118791,39490096,263534161,167501734,158758794,7515579,1


In [24]:
us_df_rates.columns

Index(['Location', 'Persons for whom poverty status determined',
       'Individuals- income below poverty level',
       'Population 16 years and over', 'In labor force', 'Employed civilians',
       'Unemployed civilians', 'us'],
      dtype='object')

In [25]:
# use .insert() to add a column constant value for year
us_df_rates.insert(1,'Year','2019')
us_df_rates

,Location,Year,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,us
0,United States,2019,320118791,39490096,263534161,167501734,158758794,7515579,1


In [26]:
us_df_rates = us_df_rates.drop(columns=['us'])
us_df_rates

,Location,Year,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians
0,United States,2019,320118791,39490096,263534161,167501734,158758794,7515579


In [27]:
us_df_rates.dtypes

Location                                      object
Year                                          object
Persons for whom poverty status determined    object
Individuals- income below poverty level       object
Population 16 years and over                  object
In labor force                                object
Employed civilians                            object
Unemployed civilians                          object
dtype: object

In [28]:
us_df_rates[['Persons for whom poverty status determined','Individuals- income below poverty level','Population 16 years and over',
           'In labor force','Employed civilians','Unemployed civilians']] = us_df_rates[['Persons for whom poverty status determined','Individuals- income below poverty level','Population 16 years and over',
           'In labor force','Employed civilians','Unemployed civilians']].apply(pd.to_numeric)
us_df_rates.dtypes

Location                                      object
Year                                          object
Persons for whom poverty status determined     int64
Individuals- income below poverty level        int64
Population 16 years and over                   int64
In labor force                                 int64
Employed civilians                             int64
Unemployed civilians                           int64
dtype: object

In [29]:
# Add a new column for unemployment rate 
us_df_rates["Unemployment rate"] = (100 * \
    us_df_rates["Unemployed civilians"] / us_df_rates["Population 16 years and over"]).round()
us_df_rates.head()

,Location,Year,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,Unemployment rate
0,United States,2019,320118791,39490096,263534161,167501734,158758794,7515579,3.0


In [30]:
# Add a new column for labor participation rate 
us_df_rates["Labor participation rate"] = (100 * \
    us_df_rates["In labor force"]/ us_df_rates["Population 16 years and over"]).round()

us_df_rates.head()

,Location,Year,Persons for whom poverty status determined,Individuals- income below poverty level,Population 16 years and over,In labor force,Employed civilians,Unemployed civilians,Unemployment rate,Labor participation rate
0,United States,2019,320118791,39490096,263534161,167501734,158758794,7515579,3.0,64.0


In [31]:
#us_df_rates = us_df_rates.loc[:,['Location','Year','Population','Pop. Black','Pop. white','Median household income','Median hh income Black','Median hh income white']]
#us_df_rates

In [32]:
# Export file as a CSV, without the Pandas index, but with the header
us_df_rates.to_csv("us_employment_2019.csv", index = False, header=True)